In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

import re
import string
import nltk
from collections import Counter
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from transformers import TFDistilBertModel,DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
uploaded = files.upload()


Saving customer_complaints_glove_v4.csv to customer_complaints_glove_v4 (2).csv


In [4]:
df= pd.read_csv('customer_complaints_glove_v4.csv')

In [5]:
df.head()

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...


In [6]:
df.isnull().sum()

product      0
narrative    0
dtype: int64

In [7]:
df1 = df

In [8]:
# Remove rows with missing data
df1.dropna(inplace=True)

# Define a function to generate sentiment labels using TextBlob
def get_sentiment(text):
    blob = TextBlob(text)
    if blob.sentiment.polarity > 0:
        return 'positive'
    elif blob.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to each row of the dataset to generate sentiment labels
df1['sentiment'] = df1['narrative'].apply(get_sentiment)


In [9]:
# Convert labels to numeric values
sentiment_dict = {'positive': 1, 'negative': 0, 'neutral': 2}
df['sentiment'] = df['sentiment'].map(sentiment_dict)

In [10]:
df1.head()

,product,narrative,sentiment
0,credit_card,purchase order day shipping amount receive pro...,1
1,credit_card,forwarded message date tue subject please inve...,0
2,retail_banking,forwarded message cc sent friday pdt subject f...,0
3,credit_reporting,payment history missing credit report speciali...,1
4,credit_reporting,payment history missing credit report made mis...,1


In [11]:
# Split the dataset into train, validation, and test sets
train_df, test_df = train_test_split(df1, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [12]:
# Tokenize text data using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(list(train_df['narrative']), truncation=True, padding=True)
val_encodings = tokenizer(list(val_df['narrative']), truncation=True, padding=True)
test_encodings = tokenizer(list(test_df['narrative']), truncation=True, padding=True)

In [13]:
# Create TensorFlow datasets from encodings and labels
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_df['sentiment'].values
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_df['sentiment'].values
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_df['sentiment'].values
))


In [14]:
# Load pre-trained DistilBERT model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [15]:
# Define training parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [16]:
# Train model on training set
history = model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=1,
    batch_size=16,
    validation_data=val_dataset.shuffle(1000).batch(16)
)

# Evaluate model on test set
model.evaluate(test_dataset.batch(16))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2031/2031 [==============================] - 631s 311ms/step - loss: 0.1454 - accuracy: 0.9451


[0.1454237848520279, 0.94507896900177]